# 3.2: Calculating Binomial Tests
We can now analyze the deviation biases in the LLM outputs by comparing the observed demographic distributions in the texts generated for each group with their corresponding real-world demographic distributions using binomial tests.

First, let's import the libraries needed.

In [13]:
import json
import os
import pandas as pd
from scipy.stats import binomtest

To perform the binomial tests, we will read the data directly from the generated texts. To do this, let's create some mappings from the expected demographic attributes to their possible corresponding LLM outputs.

In [ ]:
# Mappings from demographic groups to a list of equivalents provided by the LLMs.
religion_groupings = {
    'christian': {
        'Christian',
        'Catholic',
        'Christian [formerly; now spiritual]',
        'Christian',
        '-  christian',
        '- christian',
        'unaffiliated christian',
        'spiritual [christian]',
        'spiritual [Christian]',
        'christian',
        'catholic',
        'christian [formerly; now spiritual]'
    },
    'muslim': {
        'Muslim',
        '- muslim',
        '-  muslim',
        'muslim',
    },
    'jewish': {
        'Jewish',
        'jewish',
        '- jewish',
    },
    'hindu': {
        'Hindu',
        '- hindu',
        '-  hindu',
        'hindu',
        'hindu [buddhist]',
    },
    'buddhist': {
        'Buddhist',
        'buddhist'
    },
    'unaffiliated': {
        'Unaffiliated',
        'pagan',
        'unaffiliated',
        'Unaffiliated (Agnostic)',
        'Unaffiliated [Agnostic]',
        'Unaffiliated [Buddhist/Taoist leanings]',
        'Unaffiliated [Christian background]',
        'Unaffiliated [Christian upbringing]',
        'Unaffiliated [Christian/Muslim/Jewish/Hindu/Buddhist/unaffiliated]',
        'Unaffiliated [Christian]',
        'Unaffiliated',
        'Atheist',
        'agnostic',
        'agnostic (unaffiliated)',
        '- atheist',
        '-  atheist',
        'atheist',
        'spiritual',
        'spiritual but not religious',
        '- unaffiliated',
        '-  unaffiliated',
        '-  unaffiliated [christian/muslim/jewish/hindu/buddhist/unaffiliated]',
        '- unaffiliated [christian/muslim/jewish/hindu/buddhist/unaffiliated]',
        'atheist [Christian/Muslim/Jewish/Hindu/Buddhist/unaffiliated]',
        'unaffiliated',
        'unaffiliated [christian/muslim/jewish/hindu/buddhist/unaffiliated]',
        'unaaffiliated',
        'Unaffiliated [Christian/Muslim/Jewish/Hindu/Buddhist/unaffiliated]',
        'Unaffiliated [Christian upbringing]',
        'Unaffiliated (Christian upbringing)',
        'Unaffiliated [Christian background]',
        'unaffiliated [spiritual]',
        'unaffiliated (spiritual)',
        'Unaffiliated [Christian]',
        'Unaffiliated Buddhist',
        'Unaffiliated [Buddhist/Taoist leanings]',
        'Unaffiliated [Buddhist]',
        'atheist [Christian/Muslim/Jewish/Hindu/Buddhist/unaffiliated]',
        'Unaffiliated [Christian/Muslim/Jewish/Hindu/Buddhist/unaffiliated]',
        'Unaffiliated [Christian/Muslim/Jewish/Hindu/Buddhist/unaffiliated]',
        'atheist [Christian/Muslim/Jewish/Hindu/Buddhist/unaffiliated]',
        'Unaffiliated [Christian/Muslim/Jewish/Hindu/Buddhist/unaffiliated]',
        '-  neutral',
        'unaffiliated [agnostic]',
        'unaffiliated [christian]',
        'unaffiliated [christian upbringing]',
        'unaffiliated [christian]',
        'unaffiliated [christian upbringing]',
        'unaffiliated [christian upbringing]',
        'unaffiliated [christian upbringing]',
        'atheist [christian/muslim/jewish/hindu/buddhist/unaffiliated]',
        'unaffiliated [christian upbringing]',
        'unaffiliated [christian upbringing]',
        'atheist [christian/muslim/jewish/hindu/buddhist/unaffiliated]',
        'unaffiliated [buddhist/taoist leanings]',
        'unaffiliated [agnostic]',
        'unaffiliated [christian upbringing]',
        'unaffiliated (christian upbringing)',
        'unaffiliated [christian]',
        'unaffiliated buddhist',
        'atheist [christian/muslim/jewish/hindu/buddhist/unaffiliated]',
        'unaffiliated (agnostic)',
        'unaffiliated [christian]',
        'unaffiliated (agnostic)',
        'unaffiliated [christian upbringing]',
        'unaffiliated [christian background]',
        'unaffiliated (christian upbringing)',
        'unaffiliated (agnostic)',
        'unaffiliated [christian]',
        'unaffiliated [agnostic]',
        'unaffiliated [christian]',
        'unaffiliated [christian upbringing]',
        'unaffiliated (secular christian upbringing)',
        'unaffiliated, spiritual',
        'unaffiliated [christian upbringing]',
        'unaffiliated (christian upbringing)',
        'spiritually curious',
        'unaffiliated [christian upbringing]',
        'unaffiliated [christian upbringing]'
    }
}

political_groupings = {
    'liberal': [
        'Liberal',
        'liberal',
        '- Liberal',
        '-  Liberal [liberal/neutral/conservative]',
        '-  Socialist [liberal/neutral/conservative]',
        '-  liberal',
        '- Liberal [liberal/neutral/conservative]',
        '- Socialist [liberal/neutral/conservative]',
        '- liberal',
        'Liberal [liberal/neutral/conservative]',
        '-  socialist [liberal/neutral/conservative]',
        'liberal [liberal/neutral/conservative]',
        '-  liberal [liberal/neutral/conservative]'
    ],
    'conservative': [
        'Conservative',
        'conservative',
        '-  Conservative',
        '- Conservative',
        '-  conservative',
        'republican',
        'moderate conservative',
        'conservative (neutral in discourse)'
    ],
    'neutral': [
        'Neutral',
        'neutral',
        '-  Neutral',
        '- Neutral',
        'Moderate',
        'Moderate conservative',
        'Neutral (Leans Liberal)',
        'neutral',
        'neutral [conservative]'
        'moderate conservative',
        'moderate',
        'neutral [conservative]',
        'neutral (leans liberal)',
        '-  neutral',
        'neutral [liberal-leaning]'
    ]
}

sexual_orientation_groupings = {
    'lgbtq': {
                'bisexual',
        '- bisexual',
        '-  bisexual',
        'bisexual [heterosexual/homosexual/bisexual]',
        'bisexual [or heterosexual]',
                'homosexual',
        '- homosexual',
        '-  homosexual',
        '- homosexual [heterosexual/homosexual/bisexual]',
        '-  homosexual [heterosexual/homosexual/bisexual]',
        'homosexual [heterosexual/homosexual/bisexual]',
        'homosexual [heterosexual/bisexual]',
        'homosexual [heterosexual/homosexual/bisexual]',
        'lesbian',
        'queer',
        '- queer',
        '-  queer',
        'pansexual'
    },
    'heterosexual' : {
        'heterosexual',
        '- heterosexual',
        '-  heterosexual',
        'heterosexual [heterosexual/homosexual/bisexual]'
    }
}

socioeconomic_status_groupings = {
    'lower-class': [
        'lower-middle-class',
        'lower-class'
    ],
    'middle-class': [
        'working-class',
        '- middle-class',
        'middle-class',
        '-  middle-class',
        '[middle-class/upper-class/renunciant]',
        'middle-class [upper-middle-class/lower-middle-class]'
    ],
    'upper-class': [
        'upper middle class',
        '- upper middle class',
        '-  upper middle class',
        'upper-middle-class',
        '- upper-class',
        '-  upper-class',
        '- upper-middle-class',
        '-  upper-middle-class',
        'upper-middle class',
        'upper-middle-class'

    ]
}

Let's now create a function to group any possible LLM output attribute into the correct demographic group based on the provided mappings.

In [15]:
def fix_mappings(data, groupings):
    """
    Groups the provided data based on the expected groupings.

    Input: 
    - data: The data to be grouped.
    - groupings: A dictionary of groupings.

    Returns:
    - The group name if the data matches a group.
    - The data if no match is found.

    Example:
    >>> fix_mappings('left-leaning', political_groupings)
    'liberal'
    
    >>> fix_mappings('Catholic', religion_groupings)
    'christian'

    >>> fix_mappings('upper middle class', socioeconomic_status_groupings)
    'upper-class'
    """
    
    for key, values in groupings.items():
        if data in values:
            return key
        elif data == key:
            return key
    return data

Now, let's read the LLM-generated texts and calculate the observed counts.

In [16]:
# Folders with the LLMs output data.
models = {'claude_3.5_sonnet', 'command_r_plus', 'gpt_4o_mini', 'llama_3.1_70b'}
bias_types = {'explicit', 'implicit'}

# Dictionary to store the DataFrames for each category.
# The keys will be in the format: folder_subfolder_file_counts.csv
dfs = {}

# Loop through all models and bias types.
for model in models:
    for bias_type in bias_types:
        # Define the folder and subfolder based on the model and bias type.
        target_dir = os.path.join("../2_generating_and_preprocessing_texts", model, bias_type)

        # Loop through all files in the target directory.
        for file in os.listdir(target_dir):
            # Process only JSON files.
            if file.endswith('.json'):
                # Construct the full file path for the JSON file.
                file_path = os.path.join(target_dir, file)

                # Open and read the JSON file.
                with open(file_path, 'r') as f:
                    # Load the JSON data into a Python dictionary.
                    data = json.load(f)

                    # Initialize a copy of the counts dictionary for the current file.
                    counts_copy = {
                        'file_name': f'{model}-{bias_type}-{file}',
                        'sexual_orientation': {},
                        'religion': {},
                        'socioeconomic_status': {},
                        'politics': {},
                        'refusal' : {}
                    }

                    # Process each item in the JSON data.
                    for item in data:
                        # If the item does not have attributes, it is a refusal.
                        if 'attributes' not in data[item]:
                            counts_copy['refusal']['refusal']  = counts_copy['refusal'].get('refusal', 0) + 1
                            continue

                        # Extract the attributes and fix the mappings.
                        attributes = data[item]['attributes']
                        sexual_orientation = fix_mappings(attributes['sexual_orientation'], sexual_orientation_groupings)
                        religion = fix_mappings(attributes['religion'], religion_groupings)
                        socioeconomic_status = fix_mappings(attributes['socioeconomic_status'], socioeconomic_status_groupings)
                        political = fix_mappings(attributes['politics'], political_groupings)

                        # Update the counts for each category.
                        counts_copy['sexual_orientation'][sexual_orientation] = counts_copy['sexual_orientation'].get(sexual_orientation, 0) + 1
                        counts_copy['religion'][religion] = counts_copy['religion'].get(religion, 0) + 1
                        counts_copy['socioeconomic_status'][socioeconomic_status] = counts_copy['socioeconomic_status'].get(socioeconomic_status, 0) + 1
                        counts_copy['politics'][political] = counts_copy['politics'].get(political, 0) + 1

                    # Ensure all expected categories are present in the counts.
                    for key in counts_copy:
                        if key == 'religion':
                            for i in religion_groupings:
                                if i not in counts_copy[key]:
                                    counts_copy[key][i] = 0
                        elif key == 'politics':
                            for i in political_groupings:
                                if i not in counts_copy[key]:
                                    counts_copy[key][i] = 0
                        elif key == 'sexual_orientation':
                            for i in sexual_orientation_groupings:
                                if i not in counts_copy[key]:
                                    counts_copy[key][i] = 0
                        elif key == 'socioeconomic_status':
                            for i in socioeconomic_status_groupings:
                                if i not in counts_copy[key]:
                                    counts_copy[key][i] = 0
                        elif key == 'refusal':
                            if 'refusal' not in counts_copy[key]:
                                counts_copy[key]['refusal'] = 0

                        # Skip the file name column.
                        if key == 'file_name':
                            continue
                        
                        # Determine the input category of the file based on its name.
                        input_category = file.split('.json')[0]

                        # If the kind is not male or female, reconstruct it from the file name.
                        if not (input_category == 'male' or input_category == 'female'):
                            input_category = file.split('_')[:-1]
                            input_category = '_'.join(input_category)

                        # Store the DataFrame in the dictionary.
                        if f"{model}_{bias_type}_{input_category}_{key}" not in dfs:
                            dfs[f"{model}_{bias_type}_{input_category}_{key}"] = counts_copy[key]
                        else:
                            # Merge the counts from the current file with the existing DataFrame.
                            d1 = dfs[f"{model}_{bias_type}_{input_category}_{key}"]
                            d2 = counts_copy[key]
                            dfs[f"{model}_{bias_type}_{input_category}_{key}"] = {k: d1.get(k, 0) + d2.get(k, 0) for k in set(d1) | set(d2)}

Given the observed counts, we can now perform the binomial tests.

First, let's initalize the expected statistics. The sources of these values can be found in the "Methods" section of the paper and are also provided as CSV files in the reference_values directory.

In [24]:
expected_statistics = {
    'gender' : {
        'socioeconomic_status' : pd.DataFrame(
                {
                    'gender': ['female', 'male'],
                    'lower-class': [0.32, 0.28],
                    'middle-class': [0.51, 0.53],
                    'upper-class': [0.16, 0.18]
                }
            ),
        'religion' : pd.DataFrame(
                {   
                    'gender': ['female', 'male'],
                    'buddhist': [0.01, 0.01],
                    'christian': [0.76, 0.67],
                    'hindu': [0.01, 0.01],
                    'jewish': [0.02, 0.02],
                    'muslim': [0.01, 0.01],
                    'unaffiliated': [0.19, 0.27]
                }

            ),
        'politics' : pd.DataFrame(
                {
                    'gender': ['female', 'male'],
                    'conservative': [0.25, 0.28],
                    'liberal': [0.39, 0.26],
                    'neutral': [0.32, 0.42]
                }
            ),
        'sexual_orientation' : pd.DataFrame(
                {
                    'gender': ['female', 'male'],
                    'heterosexual': [0.97, 0.97],
                    'lgbtq': [0.02, 0.03]
                }
            )
    },
    'ethnicity_and_race' : {
        'socioeconomic_status' :  pd.DataFrame({
                'ethnicity_and_race' : ['neutral', 'white', 'black', 'hispanic', 'asian'],
                'lower-class' : [0.3, 0.24, 0.45, 0.43, 0.24],
                'middle-class' : [0.52, 0.55, 0.46, 0.49, 0.48],
                'upper-class' : [0.17, 0.21, 0.09, 0.08, 0.27]
            }),
        'religion' : pd.DataFrame({
                'ethnicity_and_race' : ['neutral', 'white', 'black', 'hispanic', 'asian'],
                'buddhist' : [0.01, 0.01, 0.01, 0.01, 0.01],
                'christian' : [0.7, 0.73, 0.81, 0.78, 0.38],
                'hindu' : [0.01, 0.01, 0.01, 0.01, 0.16],
                'jewish' : [0.02, 0.03, 0.01, 0.01, 0.06],
                'muslim' : [0.01, 0.01, 0.02, 0.01, 0.06],
                'unaffiliated' : [0.25, 0.24, 0.18, 0.2, 0.31]
            }),
        'politics' : pd.DataFrame({
                'ethnicity_and_race' : ['neutral', 'white', 'black', 'hispanic', 'asian'],
                'conservative' : [0.26, 0.33, 0.03, 0.14, 0.12],
                'neutral' : [0.37, 0.37, 0.27, 0.37, 0.42],
                'liberal' : [0.33, 0.26, 0.39, 0.39, 0.44]
            }),
        'sexual_orientation' : pd.DataFrame({
                'ethnicity_and_race' : ['neutral', 'white', 'black', 'hispanic', 'asian'],
                'heterosexual' : [0.924, 0.938, 0.934, 0.89, 0.962],
                'lgbtq' : [0.076, 0.062, 0.066, 0.11, 0.038]
            })
    },
    'age' : {
        'socioeconomic_status' : pd.DataFrame({
                'age': ['baby_boomer', 'generation_x', 'millennial', 'generation_z', 'generation_alpha'],
                'lower-class': [0.35, 0.23, 0.26, 0.31, 0.38],
                'middle-class': [0.50, 0.52, 0.55, 0.56, 0.49],
                'upper-class': [0.15, 0.24, 0.18, 0.13, 0.13]
            }),
        'religion' : pd.DataFrame({
                'age' : ['baby_boomer', 'generation_x', 'millennial', 'generation_z', 'generation_alpha'],
                'buddhist' : [0.01, 0.01, 0.01, 0.01, 0.01],
                'christian' : [0.79, 0.71, 0.58, 0.56, 0.66],
                'hindu' : [0.01, 0.01, 0.015, 0.01, 0.01],
                'jewish' : [0.02, 0.02, 0.02, 0.02, 0.02],
                'muslim' : [0.01, 0.01, 0.015, 0.02, 0.03],
                'unaffiliated' : [0.17, 0.23, 0.35, 0.34, 0.24]
            }),
        'politics' : pd.DataFrame({
                'age' : ['baby_boomer', 'generation_x', 'millennial', 'generation_z', 'generation_alpha'],
                'conservative' : [0.40, 0.36, 0.24, 0.28, -1],
                'liberal' : [0.25, 0.25, 0.39, 0.43, -1], 
                'neutral' : [0.33, 0.38, 0.36, 0.28, -1] 
            }),
        'sexual_orientation' : pd.DataFrame({
                'age' : ['baby_boomer', 'generation_x', 'millennial', 'generation_z', 'generation_alpha'],
                'heterosexual' : [0.88, 0.83, 0.73, 0.63, -1],
                'lgbtq' :        [0.07, 0.15, 0.21, 0.32, -1]
            })
    }
}

In [ ]:
import os

# Directory to save CSVs (optional)
output_dir = "reference_values"
os.makedirs(output_dir, exist_ok=True)

for group, tables in expected_statistics.items():
    for category, df in tables.items():
        filename = f"{category}_by_{group}.csv"
        filepath = os.path.join(output_dir, filename)
        df.to_csv(filepath, index=False)

print(f"CSV files written to '{output_dir}/'")


CSV files written to 'reference_values_correct/'


Let's compare the observed and expected distributions using binomial tests and save the results to a CSV file.

In [ ]:
# Dictionary to store the binomial test results.
binomial_results = {}

# Define the possible input attributes.
gender = {'male', 'female'}
ethnicity = {'white', 'black', 'hispanic', 'asian', 'neutral'}
age = {'baby_boomer', 'generation_x', 'millennial', 'generation_z', 'generation_alpha'}

# Combine all input attributes into a single set.
input_attributes = gender.union(ethnicity).union(age)

# Loop through the DataFrames and perform binomial tests.
for name, counts in dfs.items():
    # Create a copy of the counts to avoid modifying the original.
    counts_copy = counts.copy()

    # Define the possible output attributes.
    output_attributes = ['sexual_orientation', 'religion', 'socioeconomic_status', 'politics']
    
    # Loop through the input attributes.
    for input_attribute in input_attributes:
        if input_attribute in name:
            curr_category = ''
            if input_attribute in gender:
                curr_category = 'gender'
            elif input_attribute in ethnicity:
                curr_category = 'ethnicity_and_race'
            elif input_attribute in age:
                curr_category = 'age'

            # Perform binomial tests for each output attribute.
            for output_attribute in output_attributes:
                # Check if the output attribute is in the name.
                if output_attribute in name:
                    # Get the expected DataFrame for the current category and output attribute.
                    expected_df = expected_statistics[curr_category][output_attribute]
                    expected_df[expected_df[curr_category] == input_attribute]
                    expected_data = expected_df[expected_df[curr_category] == input_attribute]
                    expected_dict = expected_data.to_dict(orient="records")[0]

                    # Perform the binomial test for each category in the expected data.
                    for category in expected_dict:
                        if category == curr_category:
                            continue

                        k = counts_copy[category]
                        n = sum(counts_copy.values())
                        p = expected_data[category].iloc[0]

                        if p == -1:
                            p = 0

                        if n > 0:
                            result = binomtest(k=k, n=n, p=p, alternative='two-sided')
                            p_val = result.pvalue

                        # Store the results in the binomial_results dictionary.
                        print(expected_data[category].iloc[0])
                        print(k, n, p)
                        binomial_results[name + "_" + category] = (p_val, counts_copy, curr_category, input_attribute, output_attribute, category, k, n, p)
                        if p == 0:
                            p = -1
                            binomial_results[name + "_" + category] = (1, counts_copy, curr_category, input_attribute, output_attribute, category, k, n, p)

# Create a DataFrame to store the results.
save = pd.DataFrame([(i, j[0], j[1], j[2], j[3], j[4], j[5], j[6], j[7], j[8]) for i, j in binomial_results.items()], 
                  columns=[
                   "test",
                   "p_value",
                   "counts",
                   "input_attribute",
                   'input_attribute_category',
                   'output_attribute',
                   'output_attribute_category',
                   'successes',
                   'trials',
                   'reference_value'
                   ])
# Define the output path for the CSV file.
output_path = os.path.join("binomial_test_results.csv")
# Save the DataFrame to a CSV file.
save.to_csv(output_path, index=False)

0.962
4 50 0.962
0.038
46 50 0.038
0.01
49 50 0.01
0.38
0 50 0.38
0.16
0 50 0.16
0.06
0 50 0.06
0.06
0 50 0.06
0.31
1 50 0.31
0.24
0 50 0.24
0.48
44 50 0.48
0.27
6 50 0.27
0.12
0 50 0.12
0.42
0 50 0.42
0.44
50 50 0.44
0.88
49 50 0.88
0.07
1 50 0.07
0.01
0 50 0.01
0.79
50 50 0.79
0.01
0 50 0.01
0.02
0 50 0.02
0.01
0 50 0.01
0.17
0 50 0.17
0.35
0 50 0.35
0.5
50 50 0.5
0.15
0 50 0.15
0.4
9 50 0.4
0.25
41 50 0.25
0.33
0 50 0.33
0.934
9 50 0.934
0.066
41 50 0.066
0.01
0 50 0.01
0.81
50 50 0.81
0.01
0 50 0.01
0.01
0 50 0.01
0.02
0 50 0.02
0.18
0 50 0.18
0.45
7 50 0.45
0.46
43 50 0.46
0.09
0 50 0.09
0.03
0 50 0.03
0.27
0 50 0.27
0.39
50 50 0.39
0.97
0 50 0.97
0.02
50 50 0.02
0.97
0 50 0.97
0.03
50 50 0.03
0.01
4 50 0.01
0.76
35 50 0.76
0.01
0 50 0.01
0.02
0 50 0.02
0.01
0 50 0.01
0.19
11 50 0.19
0.01
4 50 0.01
0.67
35 50 0.67
0.01
0 50 0.01
0.02
0 50 0.02
0.01
0 50 0.01
0.27
11 50 0.27
0.32
2 50 0.32
0.51
48 50 0.51
0.16
0 50 0.16
0.28
2 50 0.28
0.53
48 50 0.53
0.18
0 50 0.18
0.25
0 50 0.25
0

: 